In [0]:
from pyspark.sql import functions as f

In [0]:
%run ./../../ingestion/structs/nb_schemas

In [0]:
# Parâmetros de entrada para a tabela Silver
src_catalog_name    = 'silver'
src_schema_name     = 'refined'
src_table_name      = 'global_superstore'

# Parâmetros da camada Gold
gold_catalog_name   = 'gold'
gold_schema_name    = 'analytics'
gold_table_prefix   = 'global_superstore'

gold_parquet_tables = '/Volumes/gold/analytics/parquets_files'

In [0]:

table_path = f"{src_catalog_name}.{src_schema_name}.{src_table_name}"
spark.sql(f"CREATE CATALOG IF NOT EXISTS {gold_catalog_name}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {gold_catalog_name}.{gold_schema_name}")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {gold_catalog_name}.{gold_schema_name}.parquets_files")
df = spark.read.table(table_path)

In [0]:
df = spark.read.table(table_path)

#### Consulta: Métricas Anuais por produto

In [0]:
# 1. Consulta: Métricas Anuais por produto
df_annual = (
    df
    .withColumn("year", f.year(f.col("order_date")))
    .groupBy("year", 'product_id')
    .agg(
        f.round(f.sum("sales"), 2).alias("total_sales"),
        f.sum("quantity").alias("total_quantity"),
        f.round(f.avg("sales"), 2).alias("avg_sales")
    )
    .orderBy("year")
)
    
(
    df_annual
    .write
    .format("delta")
    .mode("overwrite")
    .option('overwriteSchema', True)
    .saveAsTable(f"{gold_catalog_name}.{gold_schema_name}.{gold_table_prefix}_annual_metrics")
)